In [62]:
import os

In [63]:
%pwd

'f:\\Time Series Data Analysis (Upwork)\\end-to-end-time-series-data-analysis-with-mlflow'

In [64]:
#os.chdir('../')

In [65]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [66]:
from src.TimeSeriesProject .constants import *
from src.TimeSeriesProject.utils.common import read_yaml, create_directories

In [67]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [68]:
import os
from TimeSeriesProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd
from scipy import stats
from sklearn.preprocessing import MinMaxScaler

In [69]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    
    ## Note: You can add different data transformation techniques such as Scaler, PCA and all
    #You can perform all kinds of EDA in ML cycle here before passing this data to the model

    # I am only adding train_test_spliting cz this data is already cleaned up


    def train_test_spliting(self):
        data = pd.read_csv(self.config.data_path)
        jet_relevant_data = data.drop(["cycle", "op1", "op2", "op3", "sensor1", "sensor5", "sensor6","sensor9", "sensor10", "sensor16", "sensor18", "sensor19", "sensor14", "sensor13", "sensor12", "sensor11"], axis=1)

        scaler = MinMaxScaler()
        scaled_features = scaler.fit_transform(jet_relevant_data.drop(['id', 'RUL'], axis=1))
        scaled_features = pd.DataFrame(scaled_features, columns=jet_relevant_data.drop(['id', 'RUL'], axis=1).columns)

        scaled_features['id'] = jet_relevant_data['id']
        scaled_features['RUL'] = jet_relevant_data['RUL']

        dt = scaled_features.copy()

        cycle=30
        dt['label'] = dt['RUL'].apply(lambda x: 1 if x <= cycle else 0)
        
        # Split the data into training and test sets. (0.75, 0.25) split.
        train, test = train_test_split(dt)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)
        

In [70]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2023-09-13 17:56:33,202: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-09-13 17:56:33,202: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-13 17:56:33,216: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-13 17:56:33,218: INFO: common: created directory at: artifacts]
[2023-09-13 17:56:33,218: INFO: common: created directory at: artifacts/data_transformation]
[2023-09-13 17:56:34,138: INFO: 1225486524: Splited data into training and test sets]
[2023-09-13 17:56:34,138: INFO: 1225486524: (15473, 12)]
[2023-09-13 17:56:34,138: INFO: 1225486524: (5158, 12)]
(15473, 12)
(5158, 12)
